In [2]:
import tensorflow as tf
import numpy as np

# Create CNN

In [42]:
class TweetsCNN(object):
    """
    Creating a CNN model for Twitter Sentiment analysis 
    Input: Word embedding 
    Input > Convolutonal > Max-Pooling > Softmax Layer
    """
    
    def __init__(
    self, sequence_length, num_classes, vocab_size,
    embedding_size, filter_sizes, num_filters, l2_reg_lambda=0.0):
        
        # Defining Placeholders for X,y and dropout 
        self.input_x = tf.placeholder(tf.int32, [None, sequence_length], name="input_x")
        self.input_y = tf.placeholder(tf.float32, [None, num_classes], name="input_y")
        self.dropout = tf.placeholder(tf.float32, name="dropout")
        
        # L2 regularization loss
        l2_loss = tf.constant(0.0)
        
        # Define Embedding Layer
        with tf.device('/cpu:0'), tf.name_scope("embedding"):
            # Initiate a word vector
            WV = tf.Variable(
                    tf.random_uniform([vocab_size, embedding_size], -1.0,1.0))
            # Initiate embedding layer
            self.embedded_chars = tf.nn.embedding_lookup(WV, self.input_x)
            # Expand dimension to make the input 4-dimensional
            # [None, sequence_length, embedding_size, 1]
            self.embedded_chars_expanded = tf.expand_dims(self.embedded_chars, -1)
            
       
        # Convolution and Max-Pooling Layers
        pooled_outputs = []
        for i, filter_size in enumerate(filter_sizes):
            with tf.name_scope("conv-maxpool-%s" % filter_size):
                # Convolution Layer
                filter_shape = [filter_size, embedding_size, 1, num_filters]
                W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
                b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
                conv = tf.nn.conv2d(
                        self.embedded_chars_expanded,
                        W,
                        strides=[1,1,1,1],
                        padding="VALID",
                        name="conv")
                
                # Add Activation function
                activation_func = tf.nn.relu(tf.nn.bias_add(conv,b), name="relu")
                
                # Add Max-pooling layer
                pooling = tf.nn.max_pool(
                    activation_func,
                    ksize=[1, sequence_length - filter_size + 1, 1, 1],
                    strides = [1,1,1,1],
                    padding = 'VALID',
                    name="pool")
                pooled_outputs.append(pooling)
                
        # Combine all the pooled feature
        num_filters_total = num_filters * len(filter_sizes)
        self.h_pool = tf.concat(pooled_outputs,3)
        self.h_pool_flat = tf.reshape(self.h_pool, [-1, num_filters_total])
        
        # Add Dropout
        with tf.name_scope("dropout"):
            self.h_drop = tf.nn.dropout(self.h_pool_flat, self.dropout)
        
        # Predictions
        with tf.name_scope("output"):
            W = tf.get_variable(
                "W",
                shape=[num_filters_total, num_classes],
                initializer=tf.contrib.layers.xavier_initializer())
            b = tf.Variable(tf.constant(0.1, shape=[num_classes]),name = "b")
            l2_loss += tf.nn.l2_loss(W)
            l2_loss += tf.nn.l2_loss(b)
            self.scores = tf.nn.xw_plus_b(self.h_drop, W, b, name="scores")
            self.predictions = tf.argmax(self.scores, 1, name="predictions")
            
        # Define Loss Function: mean cross-entropy loss
        with tf.name_scope("loss"):
            losses = tf.nn.softmax_cross_entropy_with_logits(logits=self.scores, labels=self.input_y)
            self.loss = tf.reduce_mean(losses) + l2_reg_lambda * l2_loss
            
        # Define Accuracy
        with tf.name_scope("accuracy"):
            correct_predictions = tf.equal(self.predictions, tf.argmax(self.input_y,1))
            self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"), name="accuracy")
            
        
                
    

# Training 

In [4]:
import os 
import pandas as pd
import time
import datetime
from tensorflow.contrib import learn 
from sklearn.model_selection import train_test_split

In [5]:
dev_sample_percentage =0.1

In [6]:
# Model Hyperparameters
embedding_dim = 128
filter_sizes = [3,4,5]
num_filters = 128
dropout_keep_prob = 0.5
l2_reg_lambda = 0.0

In [8]:
# Training parameters
batch_size = 64
num_epochs = 200
evaluate_every = 100
checkpoint_every = 100
num_checkpoints = 5

In [9]:
# Other Parameters
allow_soft_placement = True
log_device_placement = False

In [10]:
# Load Data
df_model = pd.read_csv("C:/Users/Ung Lik Teng/Desktop/CodenData/Machine Learning/NLP/cleaned.csv")

In [11]:
df_model.head()

,count_words,count_mentions,count_hashtags,count_capital_words,count_excl_quest_marks,count_urls,Sentiment,clean_text,clean_count_words
0,7,0,0,1,0,0,0,sad apl friend,3
1,6,0,0,0,0,0,0,miss new moon trailer,4
2,6,0,0,0,0,0,1,omg alreadi,2
3,25,0,0,0,0,0,0,omgaga im sooo im gunna cri dentist sinc supos...,13
4,9,0,0,0,3,0,0,think mi bf cheat,4


In [12]:
x_text = df_model.clean_text
y = df_model.Sentiment

In [14]:
labels = []

for i in range(len(y)):
    if (y[i] == 1):
        labels.append([1,0])
    else:
        labels.append([0,1])

In [15]:
max_document_length = max([len(x.split(" ")) for x in x_text])
max_document_length

68

In [16]:
vocab_processor = learn.preprocessing.VocabularyProcessor(max_document_length)
x = np.array(list(vocab_processor.fit_transform(x_text)))

Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.


In [17]:
x = np.array(list(vocab_processor.fit_transform(x_text)))

In [31]:
x_train, x_test, y_train, y_test = train_test_split(x, labels, test_size=0.3, random_state=42)

In [35]:
dev_sample_index = -1 * int(dev_sample_percentage * float(len(y_train)))
x_train, x_dev = x_train[:dev_sample_index], x_train[dev_sample_index:]
y_train, y_dev = y_train[:dev_sample_index], y_train[dev_sample_index:]

In [41]:
vocab_size=len(vocab_processor.vocabulary_)
vocab_size

42915

In [46]:
#Helper Function
def train_step(x_batch, y_batch):
    """
    A single training step
    """
    feed_dict = {
        cnn.input_x: x_batch,
        cnn.input_y: y_batch,
        cnn.dropout: dropout_keep_prob
    }
            
    _, step, summaries, loss, accuracy = sess.run(
        [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy],
        feed_dict)

    time_str = datetime.datetime.now().isoformat()
    print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
    train_summary_writer.add_summary(summaries, step)

def dev_step(x_batch, y_batch, writer=None):
    """
    Evaluates model on a dev set
    """

    feed_dict = {
        cnn.input_x: x_batch,
        cnn.input_y: y_batch,
        cnn.dropout: 1.0
    }

    step, summaries, loss, accuracy = sess.run(
        [global_step, dev_summary_op, cnn.loss, cnn.accuracy],
        feed_dict)

    time_str = datetime.datetime.now().isoformat()
    print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))

    if writer:
        writer.add_summary(summaries, step)

def batch_iter(data, batch_size, num_epochs, shuffle=True):
    """
    Generates a batch iterator for a dataset.
    """

    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data

        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]

In [ ]:
with tf.Graph().as_default():
    session_conf = tf.ConfigProto(
    allow_soft_placement=allow_soft_placement,
    log_device_placement=log_device_placement)

    sess = tf.Session(config=session_conf)

    with sess.as_default():
        cnn = TweetsCNN(
            sequence_length=x_train.shape[1],
            num_classes=2,
            vocab_size=len(vocab_processor.vocabulary_),
            embedding_size=embedding_dim,
            filter_sizes=filter_sizes,
            num_filters=num_filters,
            l2_reg_lambda=l2_reg_lambda)

        # Define Training procedure
        global_step = tf.Variable(0, name="global_step", trainable=False)
        optimizer = tf.train.AdamOptimizer(1e-3)
        grads_and_vars = optimizer.compute_gradients(cnn.loss)
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

        # Keep track of gradient values and sparsity (optional)
        grad_summaries = []
        for g, v in grads_and_vars:
            if g is not None:
                grad_hist_summary = tf.summary.histogram("{}/grad/hist".format(v.name), g)
                sparsity_summary = tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
                grad_summaries.append(grad_hist_summary)
                grad_summaries.append(sparsity_summary)
        grad_summaries_merged = tf.summary.merge(grad_summaries)

        # Output directory for models and summaries
        timestamp = str(int(time.time()))
        out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
        print("Writing to {}\n".format(out_dir))

        # Summaries for loss and accuracy
        loss_summary = tf.summary.scalar("loss", cnn.loss)
        acc_summary = tf.summary.scalar("accuracy", cnn.accuracy)

        # Train Summaries
        train_summary_op = tf.summary.merge([loss_summary, acc_summary, grad_summaries_merged])
        train_summary_dir = os.path.join(out_dir, "summaries", "train")
        train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

        # Dev summaries
        dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
        dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
        dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)

        # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=num_checkpoints)

        # Write vocabulary
        vocab_processor.save(os.path.join(out_dir, "vocab"))

        # Initialize all variables
        sess.run(tf.global_variables_initializer())

        # Generate batches
        batches = batch_iter(
            list(zip(x_train, y_train)), batch_size, num_epochs)

        # Training loop. For each batch...
        for batch in batches:
            x_batch, y_batch = zip(*batch)
            train_step(x_batch, y_batch)
            current_step = tf.train.global_step(sess, global_step)
            if current_step % evaluate_every == 0:
                print("\nEvaluation:")
                dev_step(x_dev, y_dev, writer=dev_summary_writer)
                print("")

            if current_step % checkpoint_every == 0:
                path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                print("Saved model checkpoint to {}\n".format(path))

INFO:tensorflow:Summary name embedding/Variable:0/grad/hist is illegal; using embedding/Variable_0/grad/hist instead.
INFO:tensorflow:Summary name embedding/Variable:0/grad/sparsity is illegal; using embedding/Variable_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/hist is illegal; using conv-maxpool-3/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/sparsity is illegal; using conv-maxpool-3/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/b:0/grad/hist is illegal; using conv-maxpool-3/b_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/b:0/grad/sparsity is illegal; using conv-maxpool-3/b_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-4/W:0/grad/hist is illegal; using conv-maxpool-4/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-4/W:0/grad/sparsity is illegal; using conv-maxpool-4/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-4/b:0/

2018-05-23T03:18:02.751635: step 98, loss 1.04558, acc 0.546875
2018-05-23T03:18:03.113659: step 99, loss 1.26794, acc 0.484375
2018-05-23T03:18:03.424828: step 100, loss 1.17625, acc 0.515625

Evaluation:
2018-05-23T03:18:08.016544: step 100, loss 0.691035, acc 0.612659

Saved model checkpoint to C:\Users\Ung Lik Teng\Desktop\CodenData\Machine Learning\NLP\runs\1527070647\checkpoints\model-100

2018-05-23T03:18:09.496583: step 101, loss 1.41617, acc 0.46875
2018-05-23T03:18:09.809747: step 102, loss 0.819972, acc 0.65625
2018-05-23T03:18:10.144851: step 103, loss 1.10914, acc 0.578125
2018-05-23T03:18:10.466987: step 104, loss 1.13755, acc 0.59375
2018-05-23T03:18:10.813063: step 105, loss 1.19524, acc 0.53125
2018-05-23T03:18:11.137197: step 106, loss 1.13801, acc 0.515625
2018-05-23T03:18:11.447365: step 107, loss 1.02198, acc 0.640625
2018-05-23T03:18:11.754542: step 108, loss 1.18514, acc 0.640625
2018-05-23T03:18:12.060724: step 109, loss 1.0365, acc 0.6875
2018-05-23T03:18:12.37

2018-05-23T03:18:52.066155: step 219, loss 0.92082, acc 0.5625
2018-05-23T03:18:52.476058: step 220, loss 0.847682, acc 0.625
2018-05-23T03:18:52.840085: step 221, loss 0.804633, acc 0.5625
2018-05-23T03:18:53.205108: step 222, loss 0.872319, acc 0.53125
2018-05-23T03:18:53.564151: step 223, loss 0.72386, acc 0.640625
2018-05-23T03:18:53.926180: step 224, loss 0.609817, acc 0.640625
2018-05-23T03:18:54.282227: step 225, loss 0.778719, acc 0.59375
2018-05-23T03:18:54.651239: step 226, loss 0.611077, acc 0.65625
2018-05-23T03:18:55.011277: step 227, loss 0.784278, acc 0.625
2018-05-23T03:18:55.375303: step 228, loss 0.80617, acc 0.53125
2018-05-23T03:18:55.731351: step 229, loss 0.880079, acc 0.53125
2018-05-23T03:18:56.082445: step 230, loss 0.94188, acc 0.625
2018-05-23T03:18:56.443904: step 231, loss 0.88872, acc 0.5625
2018-05-23T03:18:56.814942: step 232, loss 0.949715, acc 0.578125
2018-05-23T03:18:57.171956: step 233, loss 0.850037, acc 0.59375
2018-05-23T03:18:57.529996: step 234

2018-05-23T03:19:47.743094: step 342, loss 0.802172, acc 0.59375
2018-05-23T03:19:48.090166: step 343, loss 0.787828, acc 0.625
2018-05-23T03:19:48.428262: step 344, loss 0.675425, acc 0.65625
2018-05-23T03:19:48.765364: step 345, loss 0.805906, acc 0.59375
2018-05-23T03:19:49.115424: step 346, loss 0.732399, acc 0.625
2018-05-23T03:19:49.454517: step 347, loss 0.617774, acc 0.640625
2018-05-23T03:19:49.792487: step 348, loss 0.529488, acc 0.71875
2018-05-23T03:19:50.137942: step 349, loss 0.720834, acc 0.65625
2018-05-23T03:19:50.478033: step 350, loss 0.831136, acc 0.578125
2018-05-23T03:19:50.818123: step 351, loss 0.612047, acc 0.625
2018-05-23T03:19:51.170181: step 352, loss 0.596891, acc 0.671875
2018-05-23T03:19:51.509275: step 353, loss 0.793701, acc 0.546875
2018-05-23T03:19:51.844614: step 354, loss 0.646373, acc 0.6875
2018-05-23T03:19:52.195672: step 355, loss 0.75662, acc 0.578125
2018-05-23T03:19:52.529778: step 356, loss 0.610515, acc 0.703125
2018-05-23T03:19:52.871863:

2018-05-23T03:20:50.578072: step 465, loss 0.594059, acc 0.71875
2018-05-23T03:20:50.986981: step 466, loss 0.666206, acc 0.640625
2018-05-23T03:20:51.353998: step 467, loss 0.612662, acc 0.65625
2018-05-23T03:20:51.796812: step 468, loss 0.554856, acc 0.71875
2018-05-23T03:20:52.177795: step 469, loss 0.67261, acc 0.609375
2018-05-23T03:20:52.861996: step 470, loss 0.830802, acc 0.546875
2018-05-23T03:20:53.402060: step 471, loss 0.683199, acc 0.6875
2018-05-23T03:20:53.830915: step 472, loss 0.660532, acc 0.625
2018-05-23T03:20:54.498151: step 473, loss 0.592731, acc 0.6875
2018-05-23T03:20:55.178331: step 474, loss 0.560212, acc 0.703125
2018-05-23T03:20:55.782739: step 475, loss 0.748338, acc 0.5625
2018-05-23T03:20:56.597070: step 476, loss 0.689716, acc 0.59375
2018-05-23T03:20:56.990020: step 477, loss 0.641317, acc 0.609375
2018-05-23T03:20:57.370002: step 478, loss 0.572192, acc 0.640625
2018-05-23T03:20:57.782899: step 479, loss 0.589117, acc 0.640625
2018-05-23T03:20:58.4152

...

2018-05-23T15:53:26.993856: step 21541, loss 0.0291996, acc 0.984375
2018-05-23T15:53:27.353893: step 21542, loss 0.131458, acc 0.921875
2018-05-23T15:53:27.742850: step 21543, loss 0.104547, acc 0.984375
2018-05-23T15:53:28.119877: step 21544, loss 0.134623, acc 0.96875
2018-05-23T15:53:28.475924: step 21545, loss 0.117771, acc 0.953125
2018-05-23T15:53:28.869868: step 21546, loss 0.0424899, acc 0.984375
2018-05-23T15:53:29.231901: step 21547, loss 0.044943, acc 0.984375
2018-05-23T15:53:29.609891: step 21548, loss 0.103874, acc 0.953125
2018-05-23T15:53:29.974914: step 21549, loss 0.0636703, acc 0.984375
2018-05-23T15:53:30.353900: step 21550, loss 0.0567886, acc 0.984375
2018-05-23T15:53:30.731889: step 21551, loss 0.179997, acc 0.9375
2018-05-23T15:53:31.090929: step 21552, loss 0.089835, acc 0.96875
2018-05-23T15:53:31.455951: step 21553, loss 0.114601, acc 0.953125
2018-05-23T15:53:31.845909: step 21554, loss 0.1515, acc 0.9375
2018-05-23T15:53:32.225891: step 21555, loss 0.05938

2018-05-23T15:54:22.058882: step 21660, loss 0.0786952, acc 0.953125
2018-05-23T15:54:22.512671: step 21661, loss 0.0830527, acc 0.96875
2018-05-23T15:54:22.896640: step 21662, loss 0.130244, acc 0.90625
2018-05-23T15:54:23.274629: step 21663, loss 0.039654, acc 0.984375
2018-05-23T15:54:23.664587: step 21664, loss 0.0878117, acc 0.96875
2018-05-23T15:54:24.033600: step 21665, loss 0.178126, acc 0.953125
2018-05-23T15:54:24.397627: step 21666, loss 0.140838, acc 0.984375
2018-05-23T15:54:24.765643: step 21667, loss 0.0983624, acc 0.96875
2018-05-23T15:54:25.115706: step 21668, loss 0.0507125, acc 0.984375
2018-05-23T15:54:25.462777: step 21669, loss 0.0570123, acc 0.984375
2018-05-23T15:54:25.721087: step 21670, loss 0.0427228, acc 1
2018-05-23T15:54:26.081122: step 21671, loss 0.198152, acc 0.96875
2018-05-23T15:54:26.431187: step 21672, loss 0.0823895, acc 0.96875
2018-05-23T15:54:26.800200: step 21673, loss 0.105944, acc 0.953125
2018-05-23T15:54:27.147269: step 21674, loss 0.117253

2018-05-23T15:55:23.554206: step 21779, loss 0.0833476, acc 0.953125
2018-05-23T15:55:24.401942: step 21780, loss 0.109042, acc 0.9375
2018-05-23T15:55:25.369351: step 21781, loss 0.0345961, acc 0.984375
2018-05-23T15:55:26.100396: step 21782, loss 0.117871, acc 0.953125
2018-05-23T15:55:26.800522: step 21783, loss 0.116057, acc 0.953125
2018-05-23T15:55:27.363017: step 21784, loss 0.0382121, acc 0.984375
2018-05-23T15:55:28.034226: step 21785, loss 0.0503995, acc 0.984375
2018-05-23T15:55:28.831089: step 21786, loss 0.161465, acc 0.96875
2018-05-23T15:55:29.346710: step 21787, loss 0.0813787, acc 0.96875
2018-05-23T15:55:30.089727: step 21788, loss 0.0650921, acc 0.96875
2018-05-23T15:55:30.798826: step 21789, loss 0.0392119, acc 1
2018-05-23T15:55:31.372294: step 21790, loss 0.375777, acc 0.9375
2018-05-23T15:55:31.855000: step 21791, loss 0.0449014, acc 0.984375
2018-05-23T15:55:32.574076: step 21792, loss 0.0801963, acc 0.953125
2018-05-23T15:55:33.332051: step 21793, loss 0.057942

2018-05-23T15:56:31.129429: step 21898, loss 0.0424014, acc 0.984375
2018-05-23T15:56:31.622113: step 21899, loss 0.108572, acc 0.9375
2018-05-23T15:56:32.160815: step 21900, loss 0.0974062, acc 0.96875

Evaluation:
2018-05-23T15:56:37.687993: step 21900, loss 2.30302, acc 0.709387

Saved model checkpoint to C:\Users\Ung Lik Teng\Desktop\CodenData\Machine Learning\NLP\runs\1527070647\checkpoints\model-21900

2018-05-23T15:56:39.192967: step 21901, loss 0.0646335, acc 0.96875
2018-05-23T15:56:39.598883: step 21902, loss 0.0488974, acc 0.96875
2018-05-23T15:56:40.065633: step 21903, loss 0.0604542, acc 0.96875
2018-05-23T15:56:40.557316: step 21904, loss 0.079999, acc 0.96875
2018-05-23T15:56:41.046012: step 21905, loss 0.111937, acc 0.953125
2018-05-23T15:56:41.555648: step 21906, loss 0.07285, acc 0.96875
2018-05-23T15:56:41.932637: step 21907, loss 0.0948688, acc 0.96875
2018-05-23T15:56:42.395399: step 21908, loss 0.0718235, acc 0.96875
2018-05-23T15:56:42.898055: step 21909, loss 0.

KeyboardInterrupt: 